In [ ]:
# TODO
# Set up spark session ✔
# Import single-line JSON files ✔
    # Optimize ==> Create a Function to Import all of these files and assign names ot them; perhaps using wrappers?

# Data validation
    # Check if any null values ✔
    # Customer according to specs:




## Required Imports
---

In [54]:
import pyspark 
import pyspark.pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import \
    IntegerType, \
    StringType, \
    TimestampType, \
    DoubleType


In [4]:
session = SparkSession.builder.appName('Credit Card ETL').getOrCreate()


In [5]:
session

## Extract:
---
- Customer
- Branch
- Credit


In [6]:
customer_df = session.read.load('../data/credit_card_dataset/cdw_sapp_custmer.json', format='json')

In [40]:
branch_df = session.read.load('../data/credit_card_dataset/cdw_sapp_branch.json', format='json')

In [41]:
credit_df = session.read.load('../data/credit_card_dataset/cdw_sapp_credit.json', format='json')

In [42]:
customer_df_dirty_backup = customer_df
branch_df_dirty_backup = branch_df
credit_df_dirty_backup = credit_df

## Transform
---

### Data Validation
- Ensure Correct Schema, View Current DataTypes
- Determine if any null values

In [50]:
print("Customer DataFrame:\n")
customer_df.printSchema()
print("\nBranch DataFrame:\n")
branch_df.printSchema()
print("\nCredit DataFrame:\n")
branch_df.printSchema()




Customer DataFrame:

root
 |-- APT_NO: string (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: long (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_ZIP: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- SSN: long (nullable = true)
 |-- STREET_NAME: string (nullable = true)


Branch DataFrame:

root
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_CODE: long (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_ZIP: long (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)


Credit DataFrame:

root

In [51]:
def null_checker(dataframe):
    original_count= dataframe.count()
    drop_na = dataframe.dropna()
    new_count = drop_na.count()
    if new_count != original_count:
        print(f"Original Number of Rows: {original_count} \
            \nNew Number of Rows: {new_count}\n")
    else:
        print("No nulls, no changes made\n")


In [53]:
null_checker(customer_df)
null_checker(branch_df)
null_checker(credit_df)

No nulls, no changes made
No nulls, no changes made
No nulls, no changes made


### Casting Correct Data Types

In [86]:
test = customer_df.select(customer_df['SSN'], customer_df['SSN'].cast("INT"))

In [64]:
customer_df.createOrReplaceTempView("customer_df")

In [90]:
test_2 = customer_df.withColumns({'SSN':customer_df['SSN'].cast(IntegerType()), 
                                  'CUST_ZIP':customer_df['CUST_ZIP'].cast(IntegerType()),
                                  'CUST_PHONE':customer_df['CUST_PHONE'].cast(StringType()),
                                  'LAST_UPDATED':customer_df['LAST_UPDATED'].cast(TimestampType())})

In [70]:
test_temp = session.sql(f"""
SELECT CAST(FIRST_NAME AS VARCHAR(50))
FROM customer_df """)

In [71]:
test_temp.printSchema()

root
 |-- FIRST_NAME: string (nullable = true)

